In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Project - C147/project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1-ZqOst-RT6vlWrAPZPiqbkETxbuIXYY1/Project - C147/project


In [ ]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

y_train_valid -= 769
y_test -= 769

In [ ]:
def train_data_prep(X,y,sub_sample,average,noise):

    total_X = None
    total_y = None

    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)

    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample) - resultant vec has 400 points - downsampling; performing low pass filtering on EEG signals
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)


    total_X = X_max
    total_y = y
    print('Shape of X after maxpooling:',total_X.shape)

    # Averaging + noise
    #first: mean pooling - downsampling method; smoothing
    #second: add Gaussian noise; regularizing effect
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)

    total_X = np.vstack((total_X, X_average)) #vertical stacking; 4230 samples after stacking; size of dataset increased; each of dimension 22x400
    total_y = np.hstack((total_y, y))
    print('Shape of X after averaging+noise and concatenating:',total_X.shape)

    # Subsampling + noise addition

    for i in range(sub_sample):

        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)

        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))


    print('Shape of X after subsampling and concatenating:',total_X.shape)
    print('Shape of Y:',total_y.shape)
    return total_X,total_y


def test_data_prep(X):

    total_X = None


    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)

    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, 2), axis=3)


    total_X = X_max
    print('Shape of X after maxpooling:',total_X.shape)

    return total_X

In [ ]:
#Good Result

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
## Preprocessing the dataset

X_train_valid_prep,y_train_valid_prep = train_data_prep(X_train_valid,y_train_valid,2,2,True)
X_test_prep = test_data_prep(X_test)

## Random splitting and reshaping the data
# First generating the training and validation indices using random splitting
ind_valid = np.random.choice(8460, 1000, replace=False)
ind_train = np.array(list(set(range(8460)).difference(set(ind_valid))))

# Creating the training and validation sets using the generated indices
(x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
(y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]
print('Shape of training set:',x_train.shape)
print('Shape of validation set:',x_valid.shape)
print('Shape of training labels:',y_train.shape)
print('Shape of validation labels:',y_valid.shape)


# Converting the labels to categorical variables for multiclass classification
y_train = to_categorical(y_train, 4)
y_valid = to_categorical(y_valid, 4)
y_test = to_categorical(y_test, 4)
print('Shape of training labels after categorical conversion:',y_train.shape)
print('Shape of validation labels after categorical conversion:',y_valid.shape)
print('Shape of test labels after categorical conversion:',y_test.shape)

# Adding width of the segment to be 1
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)
print('Shape of training set after adding width info:',x_train.shape)
print('Shape of validation set after adding width info:',x_valid.shape)
print('Shape of test set after adding width info:',x_test.shape)

Shape of X after trimming: (2115, 22, 800)
Shape of X after maxpooling: (2115, 22, 400)
Shape of X after averaging+noise and concatenating: (4230, 22, 400)
Shape of X after subsampling and concatenating: (8460, 22, 400)
Shape of Y: (8460,)
Shape of X after trimming: (443, 22, 800)
Shape of X after maxpooling: (443, 22, 400)
Shape of training set: (7460, 22, 400)
Shape of validation set: (1000, 22, 400)
Shape of training labels: (7460,)
Shape of validation labels: (1000,)
Shape of training labels after categorical conversion: (7460, 4)
Shape of validation labels after categorical conversion: (1000, 4)
Shape of test labels after categorical conversion: (443, 4)
Shape of training set after adding width info: (7460, 22, 400, 1)
Shape of validation set after adding width info: (1000, 22, 400, 1)
Shape of test set after adding width info: (443, 22, 400, 1)


In [ ]:
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet

# Define EEGNet model parameters
nb_classes = 4
Chans = 22
Samples = 400
dropoutRate = 0.5
kernLength = 32
F1 = 8
D = 2
F2 = 16
dropoutType = 'Dropout'

# Initialize EEGNet model
model = EEGNet(nb_classes=nb_classes, Chans=Chans, Samples=Samples,
               dropoutRate=dropoutRate, kernLength=kernLength, F1=F1, D=D, F2=F2,
               dropoutType=dropoutType)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(x=x_train, y=y_train, epochs=50, batch_size= 8, validation_data=(x_valid, y_valid))


Epoch 1/50
933/933 [==============================] - 9s 7ms/step - loss: 1.2426 - accuracy: 0.4412 - val_loss: 1.1160 - val_accuracy: 0.5320
Epoch 2/50
933/933 [==============================] - 5s 5ms/step - loss: 1.1368 - accuracy: 0.5198 - val_loss: 1.0930 - val_accuracy: 0.5460
Epoch 3/50
933/933 [==============================] - 6s 6ms/step - loss: 1.1051 - accuracy: 0.5307 - val_loss: 1.0559 - val_accuracy: 0.5780
Epoch 4/50
933/933 [==============================] - 5s 5ms/step - loss: 1.0805 - accuracy: 0.5499 - val_loss: 1.0314 - val_accuracy: 0.5790
Epoch 5/50
933/933 [==============================] - 6s 6ms/step - loss: 1.0613 - accuracy: 0.5621 - val_loss: 0.9892 - val_accuracy: 0.6180
Epoch 6/50
933/933 [==============================] - 5s 5ms/step - loss: 1.0425 - accuracy: 0.5666 - val_loss: 0.9712 - val_accuracy: 0.6180
Epoch 7/50
933/933 [==============================] - 5s 5ms/step - loss: 1.0089 - accuracy: 0.5878 - val_loss: 0.9291 - val_accuracy: 0.6500
Epoch 

In [ ]:
probs       = model.predict(x_test)
preds       = probs.argmax(axis = -1)
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

14/14 [==============================] - 0s 3ms/step
Classification accuracy: 0.702032 


### **Shallow Conv Net**

In [ ]:
# Define EEGNet model parameters
nb_classes = 4
Chans = 22
Samples = 400
dropoutRate = 0.5
kernLength = 32
F1 = 8
D = 2
F2 = 16
dropoutType = 'Dropout'
model = ShallowConvNet(nb_classes=nb_classes, Chans=Chans, Samples=Samples,
               dropoutRate=dropoutRate)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(x=x_train, y=y_train, epochs=50, batch_size= 8, validation_data=(x_valid, y_valid))

probs       = model.predict(x_test)
preds       = probs.argmax(axis = -1)
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))


Epoch 1/50
933/933 [==============================] - 7s 6ms/step - loss: 1.3589 - accuracy: 0.3929 - val_loss: 1.1417 - val_accuracy: 0.5180
Epoch 2/50
933/933 [==============================] - 5s 5ms/step - loss: 1.1068 - accuracy: 0.5334 - val_loss: 1.0175 - val_accuracy: 0.5570
Epoch 3/50
933/933 [==============================] - 4s 5ms/step - loss: 0.9750 - accuracy: 0.6051 - val_loss: 1.0424 - val_accuracy: 0.5580
Epoch 4/50
933/933 [==============================] - 5s 5ms/step - loss: 0.9087 - accuracy: 0.6389 - val_loss: 0.8029 - val_accuracy: 0.7180
Epoch 5/50
933/933 [==============================] - 6s 7ms/step - loss: 0.8586 - accuracy: 0.6639 - val_loss: 0.7687 - val_accuracy: 0.7080
Epoch 6/50
933/933 [==============================] - 5s 5ms/step - loss: 0.8332 - accuracy: 0.6767 - val_loss: 0.7947 - val_accuracy: 0.6960
Epoch 7/50
933/933 [==============================] - 5s 5ms/step - loss: 0.8138 - accuracy: 0.6854 - val_loss: 0.7704 - val_accuracy: 0.7260
Epoch 

## **DeepConvNet**

In [ ]:
model=DeepConvNet(nb_classes=nb_classes, Chans=Chans, Samples=Samples,
               dropoutRate=dropoutRate)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(x=x_train, y=y_train, epochs=50, batch_size= 8, validation_data=(x_valid, y_valid))

probs       = model.predict(x_test)
preds       = probs.argmax(axis = -1)
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

Epoch 1/50
933/933 [==============================] - 12s 9ms/step - loss: 1.5364 - accuracy: 0.3496 - val_loss: 1.7986 - val_accuracy: 0.3030
Epoch 2/50
933/933 [==============================] - 8s 8ms/step - loss: 1.2752 - accuracy: 0.4465 - val_loss: 1.3034 - val_accuracy: 0.4180
Epoch 3/50
933/933 [==============================] - 7s 8ms/step - loss: 1.1689 - accuracy: 0.4979 - val_loss: 1.1581 - val_accuracy: 0.5180
Epoch 4/50
933/933 [==============================] - 8s 8ms/step - loss: 1.1098 - accuracy: 0.5306 - val_loss: 0.9696 - val_accuracy: 0.6160
Epoch 5/50
933/933 [==============================] - 7s 7ms/step - loss: 1.0594 - accuracy: 0.5543 - val_loss: 1.0426 - val_accuracy: 0.5390
Epoch 6/50
933/933 [==============================] - 8s 8ms/step - loss: 1.0209 - accuracy: 0.5747 - val_loss: 0.9835 - val_accuracy: 0.6090
Epoch 7/50
933/933 [==============================] - 7s 7ms/step - loss: 0.9880 - accuracy: 0.5857 - val_loss: 0.9596 - val_accuracy: 0.6200
Epoch